# Import Libraries , Load Functions

In [1]:
import imports_color

Using TensorFlow backend.


# Process Data

Updated Process

In [5]:
array2 ,labels2 = load_data(data_folder="../Data/", divisor = 2,start_at = 'none',stop = '../Data/100_mp-000.png')

In [6]:
X2 = convert_img_array_to_X(array2)
y2 = labels_onehot(labels2)

In [7]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2,y2,test_size = 0.1,stratify = y2)

In [8]:
y2.shape[1]

9

# Create Model

In [9]:
model2 = Sequential()

model2.add(Conv2D(filters = 8,                   # number of filters
                         kernel_size = 5,        # height/width of filter
                         padding = "same",
                         activation='relu',      # activation function 
                         input_shape=(X_test2[0].shape))) # shape of input (dimensions of training image)

model2.add(Conv2D(16, kernel_size = 5, activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(Dropout(0.25))
 
model2.add(Conv2D(16, kernel_size = 3, activation='relu'))
model2.add(Conv2D(16, kernel_size = 3, activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))

model2.add(Flatten())
model2.add(Activation('relu'))
model2.add(Dense(y2.shape[1], activation='softmax'))

In [10]:
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 256, 400, 8)       608       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 252, 396, 16)      3216      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 126, 198, 16)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 126, 198, 16)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 124, 196, 16)      2320      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 122, 194, 16)      2320      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 61, 97, 16)        0         
__________

In [11]:
model2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
earlystop = EarlyStopping(monitor='val_acc', min_delta=0.0001, patience=3,verbose=1, mode='auto')
callbacks_list = [earlystop]

# Run Model

In [12]:
model2.fit(X_train2, y_train2, epochs=100, verbose=1, validation_data=(X_test2,y_test2),callbacks=callbacks_list)

Train on 3446 samples, validate on 383 samples
Epoch 1/100
3446/3446 [==============================] - 38s 11ms/step - loss: 1.7396 - acc: 0.3825 - val_loss: 1.0319 - val_acc: 0.6345
Epoch 2/100
3446/3446 [==============================] - 33s 10ms/step - loss: 0.7017 - acc: 0.7690 - val_loss: 0.5627 - val_acc: 0.8094
Epoch 3/100
3446/3446 [==============================] - 33s 10ms/step - loss: 0.3294 - acc: 0.8897 - val_loss: 0.4280 - val_acc: 0.8486
Epoch 4/100
3446/3446 [==============================] - 33s 10ms/step - loss: 0.1979 - acc: 0.9350 - val_loss: 0.4229 - val_acc: 0.8721
Epoch 5/100
3446/3446 [==============================] - 33s 10ms/step - loss: 0.1125 - acc: 0.9649 - val_loss: 0.3364 - val_acc: 0.9008
Epoch 6/100
3446/3446 [==============================] - 33s 10ms/step - loss: 0.0402 - acc: 0.9887 - val_loss: 0.3846 - val_acc: 0.8903
Epoch 7/100
3446/3446 [==============================] - 33s 10ms/step - loss: 0.0349 - acc: 0.9867 - val_loss: 0.4095 - val_acc: 0

In [15]:
predictions = model2.predict_classes(X_test2) 
y_test2_class= [np.where(r==1)[0][0] for r in y_test2]

confusion_matrix( y_test2_class, predictions )

array([[46,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0, 37,  2,  0,  0,  0,  7,  0,  0],
       [ 0,  1, 30,  0,  1,  0,  0,  0,  0],
       [ 0,  6,  0, 33,  2,  0,  2,  4,  0],
       [ 0,  1,  1,  0, 43,  2,  1,  0,  1],
       [ 1,  0,  0,  0,  2, 24,  1,  2,  1],
       [ 0,  2,  1,  0,  0,  0, 41,  1,  0],
       [ 0,  0,  0,  0,  0,  1,  0, 46,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0, 40]])

# Save

In [16]:
model2.save('model_03_9ppl_v1.h5')

# Transfer Learning

Load Model

In [5]:
model = load_model('model_03_9ppl_v1.h5')

In [6]:
model3 = Model(inputs=model.input, outputs=model.get_layer('flatten_1').output)

Load Data for Train , Test

In [7]:
array3 ,labels3 = load_data(data_folder="../Data/", divisor = 2,start_at = '../Data/100_mp-000.png',stop = '../Data/103_mp-000.png')

In [8]:
X3 = convert_img_array_to_X(array3)
y3 = labels_onehot(labels3)

Train-Test Split

In [9]:
X_train3, X_test3, y_train3, y_test3 = train_test_split(X3,y3,test_size = 0.1,stratify = y3)

Initialize SVM

In [11]:
from sklearn import svm
clf = svm.SVC(kernel='rbf', class_weight='balanced')

X_train3_predict = model3.predict(X_train3)
X_test3_predict = model3.predict(X_test3)

y_train3_predict = [np.where(r==1)[0][0] for r in y_train3]
y_test3_predict = [np.where(r==1)[0][0] for r in y_test3]




In [13]:
len(y_train3)
y_train3.shape

(982, 3)

In [14]:
clf.fit(X_train3_predict, y_train3_predict)

SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [15]:
predictions = clf.predict(X_test3_predict)

In [16]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test3_predict, predictions)

0.8818181818181818

In [17]:
confusion_matrix(y_test3_predict, predictions)

array([[46,  0,  0],
       [ 0, 23,  0],
       [ 0, 13, 28]])